In [17]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
from math import isnan
import datetime as dt
from datetime import datetime
from datetime import timedelta

In [18]:
file = 'cleanData_yifeng.csv'
myData = pd.read_csv(file)
myData.drop('Unnamed: 0', axis = 1,inplace = True)
def str_to_datetime (x):
    return datetime.strptime(x, '%Y-%m-%d')
myData['date'] = myData['date'].apply(str_to_datetime)

In [19]:
myData.shape

(1002789, 10)

In [20]:
myData.head(3)

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0


# Feature Engieering

In [21]:
%%time
myData['ssn'] = myData['ssn'].astype(str)
myData['firstname'] = myData['firstname'].astype(str)
myData['lastname'] = myData['lastname'].astype(str)
myData['address'] = myData['address'].astype(str)
myData['zip5'] = myData['zip5'].astype(str)
myData['dob'] = myData['dob'].astype(str)
myData['homephone'] = myData['homephone'].astype(str)
#combine two attributes 
myData['name'] = (myData['firstname']) + (myData['lastname'])
myData['address-zip5'] = (myData['address']) + (myData['zip5'])
myData['name-dob'] = (myData['name']) + (myData['dob'])

myData['name-address-zip5'] = (myData['name']) + (myData['address-zip5'])
myData['name-dob'] = (myData['name']) + (myData['dob'])
myData['name-homephone'] = (myData['name']) + (myData['homephone'])

myData['address-zip5-dob'] = (myData['address-zip5']) + (myData['dob'])
myData['address-zip5-homephone'] = (myData['address-zip5']) + (myData['homephone'])

myData['dob-homephone'] = (myData['dob']) + (myData['homephone'])

myData['homephone-name-dob'] = (myData['homephone']) + (myData['name-dob'])

#combine ssn to each attribute
myData['ssn-firstname'] = (myData['ssn']) + (myData['firstname'])
myData['ssn-lastname'] = (myData['ssn']) + (myData['lastname'])
myData['ssn-address'] = (myData['ssn']) + (myData['address'])
myData['ssn-zip5'] = (myData['ssn']) + (myData['zip5'])
myData['ssn-dob'] = (myData['ssn']) + (myData['dob'])
myData['ssn-homephone'] = (myData['ssn']) + (myData['homephone'])
myData['ssn-name'] = (myData['ssn']) + (myData['name'])
myData['ssn-address-zip5'] = (myData['ssn']) + (myData['address-zip5'])
myData['ssn-name-dob'] = (myData['ssn']) + (myData['name-dob'])


CPU times: user 8.25 s, sys: 1.37 s, total: 9.62 s
Wall time: 5.83 s


In [22]:
myData.shape

(1002789, 28)

In [23]:
myData.head(3)

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,...,homephone-name-dob,ssn-firstname,ssn-lastname,ssn-address,ssn-zip5,ssn-dob,ssn-homephone,ssn-name,ssn-address-zip5,ssn-name-dob
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,1,1797504115,0,...,1797504115XRRAMMTRSMJETJMJ1,379070012XRRAMMTR,379070012SMJETJMJ,3790700126861 EUTST PL,3790700122765,3790700121,3790700121797504115,379070012XRRAMMTRSMJETJMJ,3790700126861 EUTST PL2765,379070012XRRAMMTRSMJETJMJ1
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,...,4164239415MAMSTUJRRTTEMRRR19340615,387482503MAMSTUJR,387482503RTTEMRRR,3874825037280 URASA PL,38748250357169,38748250319340615,3874825034164239415,387482503MAMSTUJRRTTEMRRR,3874825037280 URASA PL57169,387482503MAMSTUJRRTTEMRRR19340615
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0,...,216537580SZMMUJEZSEUSEZRAE3,200332444SZMMUJEZS,200332444EUSEZRAE,2003324445581 RSREX LN,20033244456721,2003324443,200332444216537580,200332444SZMMUJEZSEUSEZRAE,2003324445581 RSREX LN56721,200332444SZMMUJEZSEUSEZRAE3


In [24]:
attributes = list(myData.columns)
tempDict = dict()
for attribute in attributes:
    #print(attribute, len(myData[attribute].unique()))
    tempDict[attribute] = len(myData[attribute].unique())
tempDF = pd.Series(tempDict, name='unique numbers')
tempDF.index.name = 'attribute'
tempDF.reset_index().sort_values('unique numbers')

,attribute,unique numbers
9,fraud_label,2
1,date,366
6,zip5,26370
3,firstname,78136
8,homephone,106755
7,dob,169240
4,lastname,177001
10,name,717126
5,address,829852
11,address-zip5,845802


In [25]:
attributes.remove('fraud_label')
attributes.remove('date')
attributes.remove('zip5')
attributes.remove('firstname')
attributes.remove('lastname')
attributes.remove('record')

In [26]:
tempDict = dict()
for attribute in attributes:
    #print(attribute, len(myData[attribute].unique()))
    tempDict[attribute] = len(myData[attribute].unique())
tempDF = pd.Series(tempDict, name='unique numbers')
tempDF.index.name = 'attribute'
tempDF.reset_index().sort_values('unique numbers')

,attribute,unique numbers
3,homephone,106755
2,dob,169240
4,name,717126
1,address,829852
5,address-zip5,845802
0,ssn,852745
13,ssn-firstname,854260
14,ssn-lastname,854301
19,ssn-name,854717
7,name-address-zip5,858196


In [27]:
%%time
#----------------------------------------------------------------------------------
#Velocity Variables
#----------------------------------------------------------------------------------
initialDF = myData.copy()
finalDF = myData.set_index('record').copy()

myData1 = initialDF.copy()
myData2 = initialDF.copy()

myData2['join_ts1'] = myData2['date']
for time_period in [0,1,3,7,30,90,180]: 
    name_end_time = 'join_ts2_' + str(time_period)
    myData2[name_end_time] = myData2['date'] + dt.timedelta(time_period)

for entity in attributes:
#for entity in ['ssn']:
    myData3 = myData1[['record', 'date', entity]]
    myData4 = myData2[[ 'join_ts1','join_ts2_0','join_ts2_1','join_ts2_3',\
                      'join_ts2_7','join_ts2_30','join_ts2_90','join_ts2_180', entity]]
    
    tempDF = pd.merge(myData3,myData4, left_on= [entity], right_on= [entity])

    for time_period in [0,1,3,7,30,90,180]: 
        name_end_time = 'join_ts2_' + str(time_period)
        tempDF2 = tempDF[(tempDF['date'] >= tempDF['join_ts1']) & (tempDF['date'] <= tempDF[name_end_time])]

        tempGroupBy = tempDF2[['record','date']].groupby('record')
       
        temp_name = str(entity) + '_' + 'count' + str(time_period) +'_'
        finalDF = pd.merge(finalDF,getattr(tempGroupBy,'count')().add_prefix(temp_name),
                      left_index=True , right_index= True, how = 'left')
        #print(temp_name)
#----------------------------------------------------------------------------------
#Velocity Change Variables
#----------------------------------------------------------------------------------
#groupbyvar_numerator = attributes
groupbyvar_denominator = attributes

days_numerator = ['0', '1']
days_denominator = [ '7', '30' ,'180']

agg__numerator = ['count']
agg__denominator = [ 'count']

for b in groupbyvar_denominator:
    for c in days_numerator:
        for d in days_denominator:
            for e in agg__numerator:
                for f in agg__denominator:
                    temp = d
                    finalDF[b + '_' + c + '_' + e + '_' + b + '_' + d + '_' + f + '_Ave'] = \
                        finalDF[b + '_' + e + c + '_date'] / \
                       (finalDF[b + '_' + f + d + '_date'] / float(temp)  )

#----------------------------------------------------------------------------------                
#Risk Table Variables
#----------------------------------------------------------------------------------
'''tempDF = finalDF[finalDF['date'] < '2016-11-01']
tempSmallOutput = tempDF['fraud_label'].sum() / tempDF['fraud_label'].count()

for entity in attributes:
#for entity in ['ssn']:

    tempName = 'RiskTable_' + str(entity)
    tempGroupBy = tempDF[[entity, 'fraud_label']].groupby(entity)
    tempCount = tempGroupBy.count()
    tempFraudSum = tempGroupBy.sum()
    
    n_mid = 10
    c = 3
    n = tempCount
    t = (n - n_mid) / c
    tempR = tempFraudSum / tempCount
    
    tempOutput = tempSmallOutput + ((tempR - tempSmallOutput ) / (1 + np.exp(-t))   )
    tempOutput = tempOutput.rename(columns={'fraud_label':tempName})
    finalDF = pd.merge(finalDF, tempOutput,
                      left_on= entity , right_on= entity, how = 'left')
    '''

CPU times: user 13min 8s, sys: 5min 39s, total: 18min 48s
Wall time: 6min 1s


In [28]:
finalDF.shape

(1002789, 313)

In [29]:
finalDF

,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,name,...,ssn-address-zip5_0_count_ssn-address-zip5_180_count_Ave,ssn-address-zip5_1_count_ssn-address-zip5_7_count_Ave,ssn-address-zip5_1_count_ssn-address-zip5_30_count_Ave,ssn-address-zip5_1_count_ssn-address-zip5_180_count_Ave,ssn-name-dob_0_count_ssn-name-dob_7_count_Ave,ssn-name-dob_0_count_ssn-name-dob_30_count_Ave,ssn-name-dob_0_count_ssn-name-dob_180_count_Ave,ssn-name-dob_1_count_ssn-name-dob_7_count_Ave,ssn-name-dob_1_count_ssn-name-dob_30_count_Ave,ssn-name-dob_1_count_ssn-name-dob_180_count_Ave
record,,,,,,,,,,,,,,,,,,,,,
1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,1,1797504115,0,XRRAMMTRSMJETJMJ,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,MAMSTUJRRTTEMRRR,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0,SZMMUJEZSEUSEZRAE,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0,SJJZSXRSZETJXTXXS,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0,SSSXUEJMSSSUUJXUZ,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
6,2016-01-01,922264214,XEEJJSTER,ERJSAXA,4322 USJXU LN,8391,19480613,5742016319,0,XEEJJSTERERJSAXA,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
7,2016-01-01,415812149,XZJRJUSRR,STSMJRUM,478 EEXUM LN,41640,19640318,7697396312,0,XZJRJUSRRSTSMJRUM,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
8,2016-01-01,373752050,EJMRRSUXR,AMTZXRU,8906 UUAJ PL,60567,19190528,2727421586,0,EJMRRSUXRAMTZXRU,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0
9,2016-01-01,769970791,RXTSZJATS,RSXMRJME,8266 SSEAR RD,37934,19900314,5520652307,0,RXTSZJATSRSXMRJME,...,180.0,7.0,30.0,180.0,7.0,30.0,180.0,7.0,30.0,180.0


In [30]:
finalDF.shape

(1002789, 313)

In [31]:
finalDF.to_csv('featureEngineering_yifeng_final3.csv')

In [16]:
finalDF

,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,name,...,RiskTable_homephone-name-dob,RiskTable_ssn-firstname,RiskTable_ssn-lastname,RiskTable_ssn-address,RiskTable_ssn-zip5,RiskTable_ssn-dob,RiskTable_ssn-homephone,RiskTable_ssn-name,RiskTable_ssn-address-zip5,RiskTable_ssn-name-dob
0,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,1,1797504115,0,XRRAMMTRSMJETJMJ,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
1,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,MAMSTUJRRTTEMRRR,...,0.042583,0.042583,0.042583,0.042583,0.042583,0.042583,0.042583,0.042583,0.042583,0.042583
2,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0,SZMMUJEZSEUSEZRAE,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
3,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0,SJJZSXRSZETJXTXXS,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
4,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0,SSSXUEJMSSSUUJXUZ,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
5,2016-01-01,922264214,XEEJJSTER,ERJSAXA,4322 USJXU LN,8391,19480613,5742016319,0,XEEJJSTERERJSAXA,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
6,2016-01-01,415812149,XZJRJUSRR,STSMJRUM,478 EEXUM LN,41640,19640318,7697396312,0,XZJRJUSRRSTSMJRUM,...,0.013454,0.013454,0.013454,0.013454,0.013454,0.013454,0.013454,0.013454,0.013454,0.013454
7,2016-01-01,373752050,EJMRRSUXR,AMTZXRU,8906 UUAJ PL,60567,19190528,2727421586,0,EJMRRSUXRAMTZXRU,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
8,2016-01-01,769970791,RXTSZJATS,RSXMRJME,8266 SSEAR RD,37934,19900314,5520652307,0,RXTSZJATSRSXMRJME,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
9,2016-01-01,732119085,EREMTZXXA,EXMMSEXT,9166 XXSAZ PL,93751,19750127,3749451123,0,EREMTZXXAEXMMSEXT,...,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706,0.013706
